In [2]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2

In [4]:
hoteles = pd.read_csv('hoteles_unicos.csv')
categorias = pd.read_csv('categorias_unicas.csv')

## 1 endpoint

In [5]:
def top_hoteles_por_ubicacion(estado:str, ciudad:str):

    estado = estado.upper()
    ciudad = ciudad.capitalize()
    
    hoteles_ubicacion = hoteles[(hoteles['state'] == estado) & (hoteles['city'] == ciudad)]
    if hoteles_ubicacion.empty:
        return {"No se encontraron hoteles en la ubicación proporcionada."}
    else:
        top_hoteles = hoteles_ubicacion.nlargest(5, 'bussines_stars')
        top_hoteles = top_hoteles[['name', 'bussines_stars']]
        return {f'El top de hoteles para tu ubicación es':top_hoteles.to_dict(orient='records')}


In [6]:
top_hoteles_por_ubicacion('Ca', 'NashviLLe')

{'El top de hoteles para tu ubicación es': [{'name': 'Homewood Suites by Hilton Nashville Vanderbilt, TN',
   'bussines_stars': 4.0},
  {'name': 'Ghost City Tours Nashville', 'bussines_stars': 4.0},
  {'name': 'Moxy Nashville Downtown', 'bussines_stars': 3.5},
  {'name': 'Kimpton Aertson Hotel', 'bussines_stars': 3.5},
  {'name': 'SpringHill Suites Marriott Nashville Downtown/Convention Center',
   'bussines_stars': 3.5}]}

## 2 endpoint

In [7]:
def distancia_haversine(lat1, lon1, lat2, lon2):
    # Convertir grados a radianes
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Radio de la Tierra en kilómetros
    radio_tierra = 6371.0

    # Diferencia de latitud y longitud
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Calcular la distancia utilizando la fórmula haversine
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distancia = radio_tierra * c

    return distancia

# Función para ordenar el DataFrame categorias en función de la cercanía al local indicado
def ordenar_por_cercania(local):
    #global hoteles, categorias
    global hoteles, categorias

    # Obtiene las coordenadas del local indicado
    latitud_local, longitud_local, ciudad_local, estado_local = hoteles[hoteles['name'] == local][['latitude', 'longitude','city','state']].values[0]

    # Filtrar las filas que tienen la misma ciudad y estado que el local
    categorias = categorias[(categorias['city'] == ciudad_local) & (categorias['state'] == estado_local)]
    
    
    # Calculamos la distancia entre las coordenadas del local y cada negocio en categorias
    categorias['distancia'] = categorias.apply(lambda row: distancia_haversine(latitud_local, longitud_local, row['latitude'], row['longitude']), axis=1)

    # Ordena categorias por la distancia calculada
    categorias_ordenado = categorias.sort_values(by='distancia')
    categorias_ordenado = categorias_ordenado[['name','categories', 'distancia', 'city', 'state']]
    
    return {f'Los locales mas cercanos son':categorias_ordenado.to_dict(orient='records')}

In [8]:
ordenar_por_cercania('Ghost City Tours Nashville')

{'Los locales mas cercanos son': [{'name': 'Country Music Hall of Fame and Museum',
   'categories': 'Arts & Entertainment, Museums',
   'distancia': 1.1397484665469897,
   'city': 'Nashville',
   'state': 'CA'},
  {'name': 'Marathon Village',
   'categories': 'Party & Event Planning, Event Planning & Services, Home Services, Food, Landmarks & Historical Buildings, Shopping Centers, Real Estate, Convenience Stores, Commercial Real Estate, Shopping, Public Services & Government, Museums, Arts & Entertainment, Venues & Event Spaces, Distilleries',
   'distancia': 1.162564736108249,
   'city': 'Nashville',
   'state': 'CA'},
  {'name': 'Sambuca Restaurant',
   'categories': 'Music Venues, American (New), Arts & Entertainment, Restaurants, Nightlife, Jazz & Blues, Bars',
   'distancia': 1.632240499927073,
   'city': 'Nashville',
   'state': 'CA'},
  {'name': 'Urban Cookhouse',
   'categories': 'American (New), Automotive, Caterers, American (Traditional), Event Planning & Services, Comfort

## 3 endpoint

In [9]:
# Función para ordenar el DataFrame categorias por 'bussines_stars' de mayor a menor, conservando la distancia
def ordenar_por_bussines_stars(local):
    global hoteles, categorias
    # Obtenemos las coordenadas del local indicado
    
    latitud_local, longitud_local, ciudad_local, estado_local = hoteles[hoteles['name'] == local][['latitude', 'longitude','city','state']].values[0]
    #latitud_local, longitud_local = hoteles[hoteles['name'] == local][['latitude', 'longitude']].values[0]
    
    # Calculamos la distancia entre las coordenadas del local y cada negocio en categorias
    categorias['distancia'] = categorias.apply(lambda row: distancia_haversine(latitud_local, longitud_local, row['latitude'], row['longitude']), axis=1)

    # Filtrar las filas que tienen la misma ciudad y estado que el local
    #categorias = categorias[(categorias['city'] == ciudad_local) & (categorias['state'] == estado_local)]

    # Ordenamos categorias por 'bussines_stars' de mayor a menor
    categorias_ordenado = categorias.sort_values(by='bussines_stars', ascending=False)

    categorias_ordenado = categorias_ordenado[['name', 'bussines_stars','categories', 'distancia', 'city', 'state']]
    return {f'El top de hoteles para tu ubicación es':categorias_ordenado.to_dict(orient='records')}

In [10]:
ordenar_por_bussines_stars('Ghost City Tours Nashville')

{'El top de hoteles para tu ubicación es': [{'name': 'Midtown Gallery Framers',
   'bussines_stars': 5.0,
   'categories': 'Art Galleries, Framing, Arts & Entertainment, Arts & Crafts, Shopping',
   'distancia': 5.929689258818121,
   'city': 'Nashville',
   'state': 'CA'},
  {'name': 'Richland Creek Greenway',
   'bussines_stars': 4.5,
   'categories': 'Hiking, Active Life, Parks',
   'distancia': 6.069372627946706,
   'city': 'Nashville',
   'state': 'CA'},
  {'name': 'Urban Cookhouse',
   'bussines_stars': 4.0,
   'categories': 'American (New), Automotive, Caterers, American (Traditional), Event Planning & Services, Comfort Food, Beer, Wine & Spirits, Parking, Sandwiches, Beer Bar, Vegetarian, Breakfast & Brunch, Wine Bars, Restaurants, Bars, Food, Nightlife',
   'distancia': 2.1364876779839155,
   'city': 'Nashville',
   'state': 'CA'},
  {'name': 'The Local',
   'bussines_stars': 4.0,
   'categories': 'Arts & Entertainment, Nightlife, Music Venues, Jazz & Blues, Bars, Restaurants, 